In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR, SPLC, NWTG
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
redz = 1.0
mstar_edges = np.logspace(11, 12, 10) * MSOL

mmbulge = holo.relations.MMBulge_Standard()
gsmf = holo.sam.GSMF_Schechter()

phi_mstar = gsmf(mstar_edges, redz)    # [Mpc^-3]
mbh_edges = mmbulge.mbh_from_mstar(mstar_edges, False)
dmdm = mmbulge.dmstar_dmbh(mstar_edges)

phi_mbh = phi_mstar * dmdm * mbh_edges / mstar_edges

xx = utils.midpoints(mstar_edges / MSOL)
yy = utils.trapz(phi_mstar, xx=np.log10(mstar_edges), cumsum=False)

fig, ax = plot.figax()
ax.plot(xx, yy, 'b-')

tw = ax.twiny()
tw.set(xscale='log', xlabel='BH Mass [Msol]')
xx = utils.midpoints(mbh_edges / MSOL)
yy = utils.trapz(phi_mbh, xx=np.log10(mbh_edges), cumsum=False)
tw.plot(xx, yy, 'r-')

plt.show()

In [ ]:
UNITS = MSOL
edges = np.log10(mbh_edges/UNITS)
print(edges)
cents = utils.midpoints(edges)
bin = 4

dist = sp.stats.norm(loc=0.0, scale=mmbulge._scatter_dex)

ll = edges[:bin+1]
cc = cents[bin]
rr = edges[bin+1:]
print("x = ", ll, cc, rr)
ll = ll - cc
rr = rr - cc
print("dp [dex] = ", ll, rr)
wl = dist.cdf(ll)
wr = dist.cdf(rr)
print("cdf = ", wl, wr)
wl = np.diff(wl)
wr = np.diff(wr)
print("pmass = ", wl, wr)

ww = dist.cdf([ll[-1], rr[0]])
print("central cdf = ", ww)
ww = np.diff(ww)
print("central mass = ", ww)

